In [17]:
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from tqdm import tqdm
import matplotlib.pyplot as plt

In [18]:
with open('ClassificationRules.txt', 'r') as file:
    class_rules=[]
    for line in file:
        class_rules.append([int(float(n)) for n in line.strip().split()])
        
class_rules=np.array(class_rules)
print(len(class_rules), len(class_rules[0])) #266 reguł klasyfikacji, każda to ciąg dł. 9350 zawierający liczby ze zbioru klas {1,2,3}

266 9350


In [19]:
with open('ImageExpertReduced.txt', 'r') as file:
    for line in file:
        exemplary=np.array([float(n) for n in line.strip().split()]) #wzorcowa klasyfikacja
print(len(exemplary))

9350


In [20]:
def binary_random(p):
    return (random.uniform(0, 1) < p)

def initial_prob_vector(d):
    return np.full(d, 0.5)

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

def random_individual(mean_vector, var, d):
    weights = [random.gauss(mean_vector[i], var) for i in range(d)]
    return softmax(weights)
    
def random_population(mean_vector, var, N,d):
    return np.array([random_individual(mean_vector, var, d) for i in range(N)])

def population_evaluation(P, f, exemplary, class_rules):
    values = [f(i, exemplary, class_rules) for i in P]
    return np.array(values)

def pbil(f,N,T,t1,t2,t3,d, exemplary, class_rules):
    f_vals=[]
    mean_vector = np.ones(d) / d 
    var = 0.001
    P = random_population(mean_vector,var,N,d)
    pop_eval = population_evaluation(P,f, exemplary, class_rules)
    
    for t in tqdm(range(T)):
        best = P[np.argmax(pop_eval)]
        mean_vector = mean_vector*(1-t1)+best*t1
        mutate_mask = np.random.rand(d) < t2
        mean_vector[mutate_mask] = mean_vector[mutate_mask] * (1 - t3) + binary_random(0.5) * t3
        P = random_population(mean_vector,var,N,d)
        pop_eval = population_evaluation(P,f, exemplary, class_rules)
        f_val=classification_quality(P[np.argmax(pop_eval)], exemplary, class_rules)
        print(f_val)
        f_vals.append(f_val)
        
    return P[np.argmax(pop_eval)], f_vals

def classification_quality(rules_weights, exemplary, class_rules):
    d=len(class_rules[0])
    vote_tab=np.zeros((3, d))
    for num, rule in enumerate(class_rules) :
        for i in range(d):
            vote_tab[rule[i]-1][i]+=rules_weights[num]
    
    classification = np.argmax(vote_tab, axis=0)+1
    return np.sum(classification == exemplary)
    
    

In [23]:
N=100
T=150
t1=0.3
t2=0.3
t3=0.2
d=266
'''
θ1 - współczynnik uczenia
θ2 - prawdopodobieństwo mutacji
θ3 - współczynnik zaburzenia podczas mutacji
'''
result, f_vals=pbil(classification_quality,N,T,t1,t2,t3,d, exemplary, class_rules)

  1%|▎                                        | 1/150 [01:21<3:21:17, 81.06s/it]

7906


  1%|▌                                        | 2/150 [02:41<3:19:05, 80.71s/it]

7945


  2%|▊                                        | 3/150 [04:01<3:17:18, 80.54s/it]

...

8001


 99%|██████████████████████████████████████▋| 149/150 [3:19:16<01:20, 80.17s/it]

8038


100%|███████████████████████████████████████| 150/150 [3:20:36<00:00, 80.25s/it]

8017


In [24]:
classification_quality(result, exemplary, class_rules)

8017

In [25]:
print(classification_quality(result, exemplary, class_rules)/9350)

0.8574331550802139


In [26]:
plt.plot(f_vals)
plt.show()


In [36]:
import numpy as np
weights=[0.3,0.2,0.5]
class_rules=[[1,2,2,3,1],[3,3,3,1,1], [3,1,2,3,3]]
exemplary=[1,3,3,1,1]

print(classification_quality(weights, exemplary, class_rules))



[[0.3 0.  0.  0.  0.3]
 [0.  0.3 0.3 0.  0. ]
 [0.  0.  0.  0.3 0. ]]
[[0.3 0.  0.  0.2 0.5]
 [0.  0.3 0.3 0.  0. ]
 [0.2 0.2 0.2 0.3 0. ]]
[[0.3 0.5 0.  0.2 0.5]
 [0.  0.3 0.8 0.  0. ]
 [0.7 0.2 0.2 0.8 0.5]]
1
